In [7]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

from bokeh.layouts import gridplot

x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

plot_options = dict(title="Linked", width=250, plot_height=250, tools='pan,wheel_zoom')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x, y1, size=10, color="firebrick")

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, **plot_options)
s3.square(x, y2, size=10, color="olive")

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

Loading BokehJS ...

In [2]:
from bokeh.models import ColumnDataSource

x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

In [3]:
import networkx as nx
from bokeh.models import Range1d, Plot, Circle, HoverTool, MultiLine
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.plotting import from_networkx

G = nx.gnm_random_graph(15, 30)

# We could use figure here but don't want all the axes and titles
plot = Plot(x_range=Range1d(-2, 2), y_range=Range1d(-2 ,2))

# Create a Bokeh graph from the NetworkX input using nx.spring_layout
graph = from_networkx(G, nx.spring_layout, scale=1.8, center=(0,0))
plot.renderers.append(graph)

# Blue circles for nodes, and light grey lines for edges
graph.node_renderer.glyph = Circle(size=25, fill_color='#2b83ba')
graph.edge_renderer.glyph = MultiLine(line_color="#cccccc", line_alpha=0.8, line_width=2)

# green hover for both nodes and edges
graph.node_renderer.hover_glyph = Circle(size=25, fill_color='#abdda4')
graph.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=4)

# When we hover over nodes, highlight adjecent edges too
graph.inspection_policy = NodesAndLinkedEdges()

plot.add_tools(HoverTool(tooltips=None))

show(plot)

In [11]:
from bokeh.io import output_notebook, show
from bokeh.plotting import ColumnDataSource, figure, output_file
from bokeh.models.tools import HoverTool
from bokeh.models import CDSView, IndexFilter
import pandas as pd
import numpy as np
data = pd.read_csv('data.csv')*[1e6,1e12]
#datad= data.to_dict();

output_notebook()

source  = ColumnDataSource(data)
indices = np.linspace(0, len(data)-1, 500, dtype=int)
view = CDSView(source=source, filters=[IndexFilter(indices)])

# Basic plot setup
p = figure(sizing_mode="stretch_width", title='Fancy hover Mie plot', x_range=[0,10], y_range=[0,45])

p.line(source=source, x="x", y="y", line_width=1, color='gray')

cr = p.circle(source=source, x='x', y='y', size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white", view=view)

p.xaxis.axis_label = "Particle radius (μm)"
p.xaxis.axis_label_text_color = "#aa6666"
p.xaxis.axis_label_text_font_size = "20px"
p.xaxis.major_label_text_font_size = "20px"
p.xaxis.axis_label_standoff = 10

p.yaxis.axis_label = "Intensity of scattered light (pW)"
p.yaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label_text_font_size = "20px"
p.yaxis.major_label_text_font_size = "20px"
p.yaxis.axis_label_standoff = 10


p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))
output_file("fancy.html", title="Fancyyyyy")
show(p)

Loading BokehJS ...

In [9]:
import numpy as np

from bokeh.layouts import column, row, gridplot
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import CustomJS, Slider, BoxSelectTool, LassoSelectTool
from bokeh.plotting import ColumnDataSource, figure, output_file, show, curdoc

output_notebook()

x1 = np.random.normal(loc=5.0, size=400) * 100
y1 = np.random.normal(loc=10.0, size=400) * 10

x2 = np.random.normal(loc=5.0, size=800) * 50
y2 = np.random.normal(loc=5.0, size=800) * 10

x3 = np.random.normal(loc=55.0, size=200) * 10
y3 = np.random.normal(loc=4.0, size=200) * 10

x = np.concatenate((x1, x2, x3))
y = np.concatenate((y1, y2, y3))

# create the horizontal histogram
hhist, hedges = np.histogram(x, bins=20)
hzeros = np.zeros(len(hedges)-1)
hmax = max(hhist)*1.1

vhist, vedges = np.histogram(y, bins=20)
vzeros = np.zeros(len(vedges)-1)
vmax = max(vhist)*1.1


TOOLS="pan,wheel_zoom,box_select,lasso_select,reset"

source  = ColumnDataSource(data=dict(x=x, y=y))
hsource = ColumnDataSource(data=dict(left=hedges[:-1], right=hedges[1:], top=hzeros, ntop=hzeros, ttop=hhist))
vsource = ColumnDataSource(data=dict(bottom=vedges[:-1], top=vedges[1:], right=vzeros, nright=vzeros, tright = vhist))

                           
# create the scatter plot
p = figure(tools=TOOLS, plot_width=600, plot_height=600, min_border=10, min_border_left=50,
           toolbar_location="above", x_axis_location=None, y_axis_location=None,
           title="Linked Histograms")
p.background_fill_color = "#fafafa"
p.select(BoxSelectTool).select_every_mousemove = False
p.select(LassoSelectTool).select_every_mousemove = False

r = p.scatter('x', 'y', source=source, size=3, color="#3A5785", alpha=0.6)



LINE_ARGS = dict(color="#3A5785", line_color=None)

ph = figure(toolbar_location=None, plot_width=p.plot_width, plot_height=200, x_range=p.x_range,
            y_range=(-hmax, hmax), min_border=10, min_border_left=50, y_axis_location="right")
ph.xgrid.grid_line_color = None
ph.yaxis.major_label_orientation = np.pi/4
ph.background_fill_color = "#fafafa"

ph.quad(source=hsource, bottom=0, left="left", right="right", top="ttop", color="white", line_color="#3A5785")
hh1 = ph.quad(source=hsource, bottom=0, left="left", right="right", top="top", alpha=0.5, **LINE_ARGS)
hh2 = ph.quad(source=hsource, bottom=0, left="left", right="right", top="ntop", alpha=0.1, **LINE_ARGS)

# create the vertical histogram

pv = figure(toolbar_location=None, plot_width=200, plot_height=p.plot_height, x_range=(-vmax, vmax),
            y_range=p.y_range, min_border=10, y_axis_location="right")
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = np.pi/4
pv.background_fill_color = "#fafafa"

pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="tright", color="white", line_color="#3A5785")
vh1 = pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="right", alpha=0.5, **LINE_ARGS)
vh2 = pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="nright", alpha=0.1, **LINE_ARGS)

layout = gridplot([[p, pv], [ph, None]], merge_tools=False)

#r.data_source.selected.indices = [1,10,20,30]

curdoc().add_root(layout)
curdoc().title = "Selection Histogram"

callback = CustomJS(args=dict(source=source, hsource=hsource, vsource=vsource),
                    code="""
    const data = source.data;
    const ids = source.selected.indices;
    const x = data['x'];
    const y = data['y'];
    const hdata = hsource.data;
    const htop = hdata['top'];
    const hntop = hdata['ntop'];
    const hleft = hdata['left'];
    const htop_total = hdata['ttop'];
    const vdata = vsource.data;
    const vright = vdata['right'];
    const vnright = vdata['nright'];
    const vbottom = vdata['bottom'];
    const vright_total = vdata['tright'];
    var hbin_size = hleft[1] - hleft[0];
    var vbin_size = vbottom[1] - vbottom[0];
    for (var i = 0; i < htop.length; i++)
    {
        htop[i] = 0;
    }
    for (var i = 0; i < vright.length; i++)
    {
        vright[i] = 0;
    }
    for (var i = 0; i < ids.length; i++) {
        var hbin = Math.floor((x[ids[i]] - hleft[0]) / hbin_size);
        var vbin = Math.floor((y[ids[i]] - vbottom[0]) / vbin_size);
        htop[hbin] = htop[hbin] + 1;
        vright[vbin] = vright[vbin] + 1;
    }
    for (var i = 0; i < hntop.length; i++)
    {
        hntop[i] = htop[i] - htop_total[i];
    }
    for (var i = 0; i < vnright.length; i++)
    {
        vnright[i] = vright[i] - vright_total[i];
    }
    source.change.emit();
    hsource.change.emit();
    vsource.change.emit();
""")


r.data_source.selected.js_on_change('indices', callback)

output_file("hist.html", title="Awesome linked histograms")
#push_notebook()
show(layout)


Loading BokehJS ...

In [10]:
import numpy as np
import pandas as pd

from bokeh.layouts import column, row, gridplot
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import CustomJS, Slider, BoxSelectTool, LassoSelectTool
from bokeh.plotting import ColumnDataSource, figure, output_file, show, curdoc

output_notebook()

data = pd.read_csv('hsr.csv')


# create the horizontal histogram
hhist, hedges = np.histogram(data['x'], bins=20)
hzeros = np.zeros(len(hedges)-1)
hmax = max(hhist)*1.1

vhist, vedges = np.histogram(data['y'], bins=20)
vzeros = np.zeros(len(vedges)-1)
vmax = max(vhist)*1.1


TOOLS="pan,wheel_zoom,box_select,lasso_select,reset"

source  = ColumnDataSource(data)
hsource = ColumnDataSource(data=dict(left=hedges[:-1], right=hedges[1:], top=hzeros, ntop=hzeros, ttop=hhist))
vsource = ColumnDataSource(data=dict(bottom=vedges[:-1], top=vedges[1:], right=vzeros, nright=vzeros, tright = vhist))

                           
# create the scatter plot
p = figure(tools=TOOLS, plot_width=600, plot_height=600, min_border=10, min_border_left=50,
           toolbar_location="above", x_axis_location=None, y_axis_location=None,
           x_range=[-0.3,4.9], y_range=[20,-2], title="Linked Histograms")
p.background_fill_color = "#fafafa"
p.select(BoxSelectTool).select_every_mousemove = False
p.select(LassoSelectTool).select_every_mousemove = False

r = p.scatter('x', 'y', source=source, size=3, color="#3A5785", alpha=0.6)



LINE_ARGS = dict(color="#3A5785", line_color=None)

ph = figure(toolbar_location=None, plot_width=p.plot_width, plot_height=200, x_range=p.x_range,
            y_range=(-hmax, hmax), min_border=10, min_border_left=50, y_axis_location="right")
ph.xgrid.grid_line_color = None
ph.xaxis.axis_label = "BP-RP colour"
ph.yaxis.major_label_orientation = np.pi/4
ph.background_fill_color = "#fafafa"

ph.quad(source=hsource, bottom=0, left="left", right="right", top="ttop", color="white", line_color="#3A5785")
hh1 = ph.quad(source=hsource, bottom=0, left="left", right="right", top="top", alpha=0.5, **LINE_ARGS)
hh2 = ph.quad(source=hsource, bottom=0, left="left", right="right", top="ntop", alpha=0.1, **LINE_ARGS)

# create the vertical histogram

pv = figure(toolbar_location=None, plot_width=200, plot_height=p.plot_height, x_range=(-vmax, vmax),
            y_range=p.y_range, min_border=10, y_axis_location="right")
pv.ygrid.grid_line_color = None
pv.yaxis.axis_label = "Absolute magnitude"
pv.xaxis.major_label_orientation = np.pi/4
pv.background_fill_color = "#fafafa"

pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="tright", color="white", line_color="#3A5785")
vh1 = pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="right", alpha=0.5, **LINE_ARGS)
vh2 = pv.quad(source=vsource, left=0, bottom="bottom", top="top", right="nright", alpha=0.1, **LINE_ARGS)

layout = gridplot([[p, pv], [ph, None]], merge_tools=False)

#r.data_source.selected.indices = [1,10,20,30]

curdoc().add_root(layout)
curdoc().title = "Selection Histogram"

callback = CustomJS(args=dict(source=source, hsource=hsource, vsource=vsource),
                    code="""
    const data = source.data;
    const ids = source.selected.indices;
    const x = data['x'];
    const y = data['y'];
    const hdata = hsource.data;
    const htop = hdata['top'];
    const hntop = hdata['ntop'];
    const hleft = hdata['left'];
    const htop_total = hdata['ttop'];
    const vdata = vsource.data;
    const vright = vdata['right'];
    const vnright = vdata['nright'];
    const vbottom = vdata['bottom'];
    const vright_total = vdata['tright'];
    var hbin_size = hleft[1] - hleft[0];
    var vbin_size = vbottom[1] - vbottom[0];
    for (var i = 0; i < htop.length; i++)
    {
        htop[i] = 0;
    }
    for (var i = 0; i < vright.length; i++)
    {
        vright[i] = 0;
    }
    for (var i = 0; i < ids.length; i++) {
        var hbin = Math.floor((x[ids[i]] - hleft[0]) / hbin_size);
        var vbin = Math.floor((y[ids[i]] - vbottom[0]) / vbin_size);
        htop[hbin] = htop[hbin] + 1;
        vright[vbin] = vright[vbin] + 1;
    }
    for (var i = 0; i < hntop.length; i++)
    {
        hntop[i] = htop[i] - htop_total[i];
    }
    for (var i = 0; i < vnright.length; i++)
    {
        vnright[i] = vright[i] - vright_total[i];
    }
    source.change.emit();
    hsource.change.emit();
    vsource.change.emit();
""")


r.data_source.selected.js_on_change('indices', callback)

output_file("hsr_hist.html", title="Awesome linked HSR")
#push_notebook()
show(layout)


Loading BokehJS ...